In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar os datasets
fato_vendas = pd.read_csv(r"C:\Users\alexm\Documents\Altice_DataAnalyst\fato_vendas.csv", sep=';', decimal=',')
dim_clientes = pd.read_csv(r"C:\Users\alexm\Documents\Altice_DataAnalyst\dim_clientes.csv", sep=';', decimal=',')
dim_cidades = pd.read_csv(r"C:\Users\alexm\Documents\Altice_DataAnalyst\dim_cidades.csv", sep=';', decimal=',')
dim_produtos = pd.read_csv(r"C:\Users\alexm\Documents\Altice_DataAnalyst\dim_produtos.csv", sep=';', decimal=',')

# Remover espaços extras dos nomes das colunas
fato_vendas.columns = fato_vendas.columns.str.strip()
dim_clientes.columns = dim_clientes.columns.str.strip()
dim_cidades.columns = dim_cidades.columns.str.strip()
dim_produtos.columns = dim_produtos.columns.str.strip()

# Exibir nomes das colunas para verificar
print("Colunas em fato_vendas:", fato_vendas.columns)
print("Colunas em dim_cidades:", dim_cidades.columns)
print("Colunas em dim_produtos:", dim_produtos.columns)

# Exibir informações gerais
def resumo_dados(df, nome):
    print(f"\nResumo do dataset: {nome}")
    print("Linhas:", df.shape[0])
    print("Colunas:", df.shape[1])
    print(df.dtypes)
    print("-" * 40)

# Mostrar resumo de cada tabela
resumo_dados(fato_vendas, "Fato Vendas")
resumo_dados(dim_clientes, "Dim Clientes")
resumo_dados(dim_cidades, "Dim Cidades")
resumo_dados(dim_produtos, "Dim Produtos")

# Verificar se a coluna 'Valor_Euro' existe
if 'Valor_Euro' in fato_vendas.columns:
    def estatisticas(df, coluna):
        print(f"\nEstatísticas da coluna {coluna}:")
        print(df[coluna].describe())
        print("-" * 40)
    
    estatisticas(fato_vendas, "Valor_Euro")
else:
    print("A coluna 'Valor_Euro' não foi encontrada em fato_vendas. Verifique o nome correto.")

# Visualização: Histograma de Vendas
if 'Valor_Euro' in fato_vendas.columns:
    plt.figure(figsize=(10, 5))
    sns.histplot(fato_vendas["Valor_Euro"], bins=30, kde=True)
    plt.title("Distribuição dos Valores de Vendas")
    plt.xlabel("Valor em Euros")
    plt.ylabel("Frequência")
    plt.show()

# Ajustar nomes de colunas para o merge
col_fato_vendas_cidade = 'cidade_id' if 'cidade_id' in fato_vendas.columns else 'CidadeID'
col_fato_vendas_produto = 'produto_id' if 'produto_id' in fato_vendas.columns else 'ProdutoID'
col_dim_cidades = 'cidade_id' if 'cidade_id' in dim_cidades.columns else 'CidadeID'
col_dim_produtos = 'produto_id' if 'produto_id' in dim_produtos.columns else 'ProdutoID'

# Merge para análises
fato_vendas = fato_vendas.merge(dim_cidades, left_on=col_fato_vendas_cidade, right_on=col_dim_cidades)
fato_vendas = fato_vendas.merge(dim_produtos, left_on=col_fato_vendas_produto, right_on=col_dim_produtos)

# Verificar se as colunas de cidade e produto existem antes de plotar
def plot_bar(df, x_col, y_col, title, palette):
    if x_col in df.columns and y_col in df.columns:
        plt.figure(figsize=(12, 6))
        ax = sns.barplot(x=df[x_col], y=df[y_col], estimator=sum, palette=palette)
        plt.xticks(rotation=45)
        plt.title(title)
        plt.ylabel("Total de Vendas (€)")

        # Adicionar rótulos
        for p in ax.patches:
            altura = p.get_height()
            ax.annotate(f'{altura:,.0f}',
                        (p.get_x() + p.get_width() / 2., altura),
                        ha='center', va='bottom', fontsize=10, color='black',
                        xytext=(0, 5), textcoords='offset points')
        plt.show()
    else:
        print(f"Colunas {x_col} ou {y_col} não encontradas no dataset.")

plot_bar(fato_vendas, "Cidade", "Valor_Euro", "Vendas Totais por Cidade", "viridis")
plot_bar(fato_vendas, "Produto", "Valor_Euro", "Vendas Totais por Produto", "coolwarm")

# Evolução das Vendas ao longo do tempo
if 'data_venda' in fato_vendas.columns:
    fato_vendas["data_venda"] = pd.to_datetime(fato_vendas["data_venda"], errors='coerce')
    fato_vendas.dropna(subset=["data_venda"], inplace=True)  # Remover valores inválidos
    fato_vendas["Ano_Mes"] = fato_vendas["data_venda"].dt.to_period("M")
    vendas_por_mes = fato_vendas.groupby("Ano_Mes")["Valor_Euro"].sum().reset_index()
    vendas_por_mes["Ano_Mes"] = vendas_por_mes["Ano_Mes"].astype(str)

    plt.figure(figsize=(12, 6))
    ax = sns.lineplot(data=vendas_por_mes, x="Ano_Mes", y="Valor_Euro", marker="o")
    plt.title("Evolução das Vendas ao Longo do Tempo")
    plt.ylabel("Total de Vendas (€)")
    plt.xlabel("Ano-Mês")
    plt.xticks(rotation=45)

    # Adicionar rótulos
    for i, row in vendas_por_mes.iterrows():
        ax.annotate(f'{row["Valor_Euro"]:,.0f}', (i, row["Valor_Euro"]), textcoords="offset points", xytext=(0,10), ha='center')
    plt.show()
else:
    print("A coluna 'data_venda' não foi encontrada em fato_vendas.")

print("EDA concluída com sucesso!")

Colunas em fato_vendas: Index(['venda_id', 'cliente_id', 'produto_id', 'data_venda', 'quantidade',
       'preco_unitario', 'total_venda'],
      dtype='object')
Colunas em dim_cidades: Index(['cidade_id', 'cidade', 'populacao'], dtype='object')
Colunas em dim_produtos: Index(['produto_id', 'produto', 'min', 'max'], dtype='object')

Resumo do dataset: Fato Vendas
Linhas: 50000
Colunas: 7
venda_id            int64
cliente_id          int64
produto_id          int64
data_venda         object
quantidade          int64
preco_unitario    float64
total_venda       float64
dtype: object
----------------------------------------

Resumo do dataset: Dim Clientes
Linhas: 10000
Colunas: 5
cliente_id     int64
nome          object
email         object
telefone      object
cidade_id      int64
dtype: object
----------------------------------------

Resumo do dataset: Dim Cidades
Linhas: 10
Colunas: 3
cidade_id     int64
cidade       object
populacao     int64
dtype: object
--------------------------

KeyError: 'CidadeID'

In [4]:
print("Colunas em fato_vendas:", fato_vendas.columns)
print("Colunas em dim_cidades:", dim_cidades.columns)
print("Colunas em dim_produtos:", dim_produtos.columns)


Colunas em fato_vendas: Index(['venda_id', 'cliente_id', 'produto_id', 'data_venda', 'quantidade',
       'preco_unitario', 'total_venda'],
      dtype='object')
Colunas em dim_cidades: Index(['cidade_id', 'cidade', 'populacao'], dtype='object')
Colunas em dim_produtos: Index(['produto_id', 'produto', 'min', 'max'], dtype='object')


In [7]:
# Exibir informações detalhadas de cada DataFrame

dataframes = {
    "Fato Vendas": fato_vendas,
    "Dim Clientes": dim_clientes,
    "Dim Cidades": dim_cidades,
    "Dim Produtos": dim_produtos
}

for nome, df in dataframes.items():
    print(f"--- {nome} ---")
    print("Colunas:", df.columns.tolist())
    print("Tipos:")
    print(df.dtypes)
    print("Shape:", df.shape)
    print("Primeiras linhas:")
    print(df.head(), "\n")
    print("-" * 40, "\n")

--- Fato Vendas ---
Colunas: ['venda_id', 'cliente_id', 'produto_id', 'data_venda', 'quantidade', 'preco_unitario', 'total_venda']
Tipos:
venda_id            int64
cliente_id          int64
produto_id          int64
data_venda         object
quantidade          int64
preco_unitario    float64
total_venda       float64
dtype: object
Shape: (50000, 7)
Primeiras linhas:
   venda_id  cliente_id  produto_id  data_venda  quantidade  preco_unitario  \
0         1        9486           2  2022-06-25           1           68.33   
1         2        6184           6  2022-05-14           1           84.34   
2         3        7241           1  2023-09-10           3           42.52   
3         4        1078           2  2022-04-25           3           51.04   
4         5        6250           7  2022-07-16           3          425.59   

   total_venda  
0        68.33  
1        84.34  
2       127.56  
3       153.12  
4      1276.77   

---------------------------------------- 

--- Dim 